In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import prophet

#city indexes: Adilabad->0,Karimnagar->1,Khammam->2,Nizamabad->3,Warangal->4

if(index==0):
    Df=pd.read_csv("Adilabad/city_combined.csv")
elif(index==1):
    Df=pd.read_csv("Karimnagar/city_combined.csv")
elif(index==2):
    Df=pd.read_csv("Khammam/city_combined.csv")
elif(index==3):
    Df=pd.read_csv("Nizamabad/city_combined.csv")
elif(index==4):
    Df=pd.read_csv("Warangal/city_combined.csv")

Df.drop('temp_min (⁰C)',axis=1,inplace=True)


Df['windspeed(Kmph)']=(Df['wind_speed_max (Kmph)']+Df['wind_speed_min (Kmph)'])/2
Df['Humidity(%)']=(Df['Humidity_max (%)']+Df['humidity_min (%)'])/2
Df.drop(['wind_speed_min (Kmph)','humidity_min (%)','Humidity_max (%)','wind_speed_max (Kmph)'],axis=1,inplace=True)

Df['windspeed(Kmph)'].fillna(value=Df['windspeed(Kmph)'].mean(),inplace=True)

lockdowns = pd.DataFrame([
    {'holiday': 'lockdown_1', 'ds': '2020-03-22', 'lower_window': 0, 'ds_upper': '2020-05-31'},
    {'holiday': 'lockdown_2', 'ds': '2021-05-12', 'lower_window': 0, 'ds_upper': '2021-05-22'}])
for t_col in ['ds', 'ds_upper']:
    lockdowns[t_col] = pd.to_datetime(lockdowns[t_col])
lockdowns['upper_window'] = (lockdowns['ds_upper'] - lockdowns['ds']).dt.days

m=prophet.Prophet(growth='logistic',holidays=lockdowns,yearly_seasonality=True)
m.add_country_holidays(country_name='IN')
Df['ds']=Df['Date']
Df['ds'] = pd.to_datetime(Df['ds'])
Df['y']=Df['temp_max (⁰C)']
cap=Df['temp_max (⁰C)'].max()-2
Df['cap']=cap
Df.drop(["Date","temp_max (⁰C)"],axis=1,inplace=True)
for i in ['Rainfall (mm)','windspeed(Kmph)',
       'Humidity(%)']:
    m.add_regressor(i)

m.fit(Df)
future = m.make_future_dataframe(periods=365)
today=[0,25,75,0.25,7.88]
j=0
for i in ['Rainfall (mm)','windspeed(Kmph)',
       'Humidity(%)']:
    future[i]=today[j]
    j+=1
future['cap']=cap
forecast=m.predict(future)
forecast=forecast[['ds','yhat','yhat_lower','yhat_upper']]

NameError: name 'index' is not defined